In [105]:
import requests
import sqlite3
from bs4 import BeautifulSoup
import MeCab as mc
import gensim
from gensim.models import Word2Vec
from gensim import matutils
from gensim.corpora.dictionary import Dictionary
from sklearn import ensemble
from sklearn.model_selection import train_test_split

In [64]:

conn = None

#  データベースに接続する
def connect():
    global conn
    conn = sqlite3.connect('./yahoonews.db')

def close():
    conn.close()
    
def create_table():
    conn.execute('DROP TABLE IF EXISTS news')
    conn.execute("""CREATE TABLE news (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title text,
            category text
            )""")

#  Countriesテーブルにレコードを挿入
def load(title, category):
    conn.execute(
        'INSERT INTO news (title, category) VALUES (?, ?)', (title, category))
    conn.commit()
    
def get_all_title(limit, offset=0):
    return [record[0] for record in 
               conn.execute(
            'SELECT title FROM news LIMIT ? OFFSET ?',
            (limit, offset))]

In [60]:
connect()
create_table()
close()

In [ ]:
url = "https://news.yahoo.co.jp/"

r  = requests.get(url)
bs = BeautifulSoup(r.text, "html.parser")

In [ ]:
p    = bs.find("p", class_="topicsList_button_more")
link = p.find("a").get("href")

r = requests.get(link)
bs = BeautifulSoup(r.text, "html.parser")
bs

In [ ]:
category_ul = bs.find("ul", class_="cateTab")
category_list = category_ul.find_all("a")
sports_link = category_list[5].get("href")
it_link = category_list[6].get("href")

In [61]:
# 欲しいカテゴリーのurlを渡すとリンク先から1000件分のニュースのタイトルの文字列リストを返す
def get_title_list(url):
    header = "https://news.yahoo.co.jp"
    title_list = []
    for i in range(50):
        r  = requests.get(header + url)
        bs = BeautifulSoup(r.text , "html.parser")
        
        dl_list = bs.find_all("dl", class_="title")
        page_list = []
        for dl in dl_list:
            page_list.append(dl.dt.text)
        title_list += page_list    
        
        url = bs.find("li", class_="next")
        url = url.find("a").get("href")
    return title_list
    
def load_list(title_list, category):
    for title in title_list:
        load(title, category)

In [62]:
sports_title = get_title_list(sport_link)
it_title = get_title_list(it_link)

connect()
load_list(sports_title, "スポーツ")
load_list(it_title, "IT")
close()

In [116]:
def mecab_analysis(texts):
    t = mc.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")
    t.parse('')
    output = []
    node =  t.parseToNode(texts)  
    while node:
        if node.surface != "":  # ヘッダとフッタを除外
            word_type = node.feature.split(",")[0]
            if word_type in ['名詞', "形容詞", "動詞"]:
                output.append(node.surface)
        node = node.next
        if node is None:
            break
    return output

In [117]:
connect()
title_list = get_all_title(-1)

wakati_list = [mecab_analysis(title) for title in title_list]

In [68]:
wakati_list

[['マー君', 'ももクロ', '観戦', '敗戦'],
 ['平尾誠二', 'さん', '娘', '父'],
 ['U', '20日', '大人', 'サッカー', '必要'],
 ['大谷', 'MLB', '球宴', 'DH', '候補', '入り'],
 ['ももクロ', 'マー君', '観戦', '興奮'],
 ['松坂', '来月21日', '1軍', '復帰'],
 ['大坂', '初戦突破', '人生', '一番', '緊張'],
 ['仏', '大', '坂', '大逆転', '初戦突破'],
 ['幻', '80年', '五輪', '代表', '笹田', '氏', '死去'],
 ['仏', 'OP', '大坂なおみ', '1回戦'],
 ['八百長', 'レアル', 'DF', 'ら', '逮捕'],
 ['浦和', '監督解任', '後任', '組長'],
 ['羽生結弦', '確か', '回復'],
 ['川内優輝', '世界陸上', '代表復帰'],
 ['青木宣親', '丸刈り', '体', '鼓舞'],
 ['中日', '松坂', '257日', 'ぶり', '実戦', '登板'],
 ['188cm', '下手', 'SB', '右腕'],
 ['相撲', '座布団', '罪'],
 ['大坂', '赤土', '克服'],
 ['大谷', '技あり', '安打', '二', '盗', '失敗'],
 ['五輪', 'チケ', '申し込み', 'きょう'],
 ['ドラフト', '指名', '優先', '権', '変更'],
 ['2軍', '戦', '100', '球', '輝', '星', '昇格', '条件'],
 ['横審', '混乱', '審判', '部', '苦言'],
 ['全豪', 'V', 'クビトバ', '仏', '棄権'],
 ['広島', '浮上', 'きっかけ', '丸', '2ラン'],
 ['広島', '浮上', '4月', '丸', '2ラン', '覚醒'],
 ['夏場所', '大統領', '視聴率', '26.3%'],
 ['単勝', '93倍', '伏兵', 'ダービー', 'V'],
 ['朝乃', '山', '親方', 'クーラー'],
 ['何', '本田圭佑'],
 ['マエケン', 

In [72]:
model = Word2Vec(wakati_list, size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

In [83]:
# sim = model.similarity("マー君", "イチロー")

synonym = model.most_similar(positive="ももクロ")
synonym

/home/y-watanabe/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


[('洞窟', 0.35756874084472656),
 ('高田', 0.3421747386455536),
 ('空振り', 0.3156339228153229),
 ('安楽死', 0.31274357438087463),
 ('発売前', 0.30569547414779663),
 ('アウト', 0.2991495132446289),
 ('燕', 0.29906147718429565),
 ('江東区', 0.2972562611103058),
 ('銀行', 0.29091835021972656),
 ('爆破予告', 0.2873806953430176)]

In [87]:
dictionary = Dictionary(wakati_list)
print(dictionary.token2id)

{'ももクロ': 0, 'マー君': 1, '敗戦': 2, '観戦': 3, 'さん': 4, '娘': 5, '平尾誠二': 6, '父': 7, '20日': 8, 'U': 9, 'サッカー': 10, '大人': 11, '必要': 12, 'DH': 13, 'MLB': 14, '候補': 15, '入り': 16, '大谷': 17, '球宴': 18, '興奮': 19, '1軍': 20, '復帰': 21, '来月21日': 22, '松坂': 23, '一番': 24, '人生': 25, '初戦突破': 26, '大坂': 27, '緊張': 28, '仏': 29, '坂': 30, '大': 31, '大逆転': 32, '80年': 33, '五輪': 34, '代表': 35, '幻': 36, '死去': 37, '氏': 38, '笹田': 39, '1回戦': 40, 'OP': 41, '大坂なおみ': 42, 'DF': 43, 'ら': 44, 'レアル': 45, '八百長': 46, '逮捕': 47, '後任': 48, '浦和': 49, '監督解任': 50, '組長': 51, '回復': 52, '確か': 53, '羽生結弦': 54, '世界陸上': 55, '代表復帰': 56, '川内優輝': 57, '丸刈り': 58, '体': 59, '青木宣親': 60, '鼓舞': 61, '257日': 62, 'ぶり': 63, '中日': 64, '実戦': 65, '登板': 66, '188cm': 67, 'SB': 68, '下手': 69, '右腕': 70, '座布団': 71, '相撲': 72, '罪': 73, '克服': 74, '赤土': 75, '二': 76, '失敗': 77, '安打': 78, '技あり': 79, '盗': 80, 'きょう': 81, 'チケ': 82, '申し込み': 83, 'ドラフト': 84, '優先': 85, '変更': 86, '指名': 87, '権': 88, '100': 89, '2軍': 90, '戦': 91, '昇格': 92, '星': 93, '条件': 94, '球': 95, '輝': 96, '審判': 97,

In [88]:
vec = dictionary.doc2bow(['本田圭佑', 'メルボルン', 'V', '退団', '発表'])
print(vec)

[(102, 1), (126, 1), (386, 1), (387, 1), (388, 1)]


In [96]:
dense = []
for wakati in wakati_list:
    vec = dictionary.doc2bow(wakati)
    dense.append(list(matutils.corpus2dense([vec], num_terms=len(dictionary)).T[0]))

In [100]:
dense[0][0]

1.0

In [114]:
X = dense
Y = [0 if i < 1000 else 1 for i in range(2000)]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8)

In [115]:
random_forest = ensemble.RandomForestClassifier(max_depth=10)
random_forest.fit(X_train, Y_train)
random_forest.score(X_test, Y_test)

/home/y-watanabe/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.65